In [ ]:
%pip install wand
!apt-get install libmagickwand-dev

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
np.random.seed(2)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Generate training dataset
T = 20
L = 1000
N = 100

x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
data = np.sin(x / 1.0 / T).astype('float64')

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

class Sequence(nn.Module):
  """
  Define LTSM model to predict. There are 2 LSTM layers and 1 linear layer
  input -> LSTM -> LSTM -> Linear -> output
  Output of LSTM haing size 51 to capture the frequency of sine function,
  then these features are pushed into linear layer to predict 1 next value
  """
  def __init__(self):
    super(Sequence, self).__init__()
    self.lstm1 = nn.LSTMCell(1, 51)
    self.lstm2 = nn.LSTMCell(51, 51)
    self.linear = nn.Linear(51, 1)
  
  def forward(self, input, future = 0):
      outputs = []
      h_t = torch.zeros(input.size(0), 51, dtype=torch.double).to(device)
      c_t = torch.zeros(input.size(0), 51, dtype=torch.double).to(device)
      h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double).to(device)
      c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double).to(device)

      for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
          h_t, c_t = self.lstm1(input_t, (h_t, c_t))
          h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
          output = self.linear(h_t2)
          outputs += [output]
      for i in range(future):# if we should predict the future
          h_t, c_t = self.lstm1(output, (h_t, c_t))
          h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
          output = self.linear(h_t2)
          outputs += [output]
      outputs = torch.stack(outputs, 1).squeeze(2)
      return outputs

In [ ]:
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)
# load data and make training set
input = torch.from_numpy(data[3:, :-1]).to(device)
target = torch.from_numpy(data[3:, 1:]).to(device)
test_input = torch.from_numpy(data[:3, :-1]).to(device)
test_target = torch.from_numpy(data[:3, 1:]).to(device)

In [ ]:
input.shape, target.shape

(torch.Size([97, 999]), torch.Size([97, 999]))

In [ ]:
# build the model
seq = Sequence().to(device)
seq.double()
criterion = nn.MSELoss()
# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
#begin to train
for i in range(9):
    print('STEP: ', i)
    def closure():
        optimizer.zero_grad()
        out = seq(input)
        loss = criterion(out, target)
        print('loss:', loss.item())
        loss.backward()
        return loss
    optimizer.step(closure)
    # begin to predict, no need to track gradient here
    with torch.no_grad():
        future = 1000
        pred = seq(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print('test loss:', loss.item())
        y = pred.cpu().detach().numpy()

STEP:  0
loss: 0.5232618273269376
loss: 0.5102756077161258
loss: 0.4808570375818254
loss: 0.468535563280216
loss: 0.4259845377799631
loss: 0.5157926443824742
loss: 0.2704566437379761
loss: 0.11931644973071746
loss: 3.4599193733829376
loss: 0.043432640399244035
loss: 0.029299380139148262
loss: 0.02449449696043418
loss: 0.02377544421986648
loss: 0.023092435314603615
loss: 0.02153848495984922
loss: 0.018752435249141223
loss: 0.014304672296588503
loss: 0.006829892151179761
loss: 0.0055648657380485056
loss: 0.004680194990260141
test loss: 0.0025366800830563647
STEP:  1
loss: 0.002596144268814951
loss: 0.0020321605738842577
loss: 0.0018123996560673742
loss: 0.0017216475960275916
loss: 0.001547140222974879
loss: 0.0012611929244023745
loss: 0.0009823830180296562
loss: 0.0008383725311573748
loss: 0.00083007702211389
loss: 0.000814618357658919
loss: 0.0007883118096878153
loss: 0.0007370170318109564
loss: 0.0006537767217807537
loss: 0.0005423691258222698
loss: 0.0004326284270049619
loss: 0.000396

In [ ]:
# draw the result
plt.figure(figsize=(30,10))
plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
plt.xlabel('x', fontsize=20)
plt.ylabel('y', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
def draw(yi, color):
      plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
      plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
draw(y[0], 'r')
draw(y[1], 'g')
draw(y[2], 'b')
plt.savefig('predict%d.pdf'%i)
plt.close()

In [ ]:
from wand.image import Image as WImage
img = WImage(filename='predict8.pdf')
img

PolicyError: ignored